In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
import collections
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.models as models
import pandas as pd
import torch.nn.functional as F
import random
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
import math

C:\Users\user\.conda\envs\pytorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:25: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [2]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
random_seed = 1024

In [4]:
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

In [5]:
data_root_lv = 'X:\\jechoi\\numpy\\Radiogenomics+genomics_raiomics_LAUD+LSCC\\train_val_test\\img\\numpy'

In [6]:
training_latent_variable_raw = np.load(data_root_lv + '\\training_autoencoder_input.npy')
training_label_bf = np.load(data_root_lv + '\\training_autoencoder_output_stage.npy')
validation_latent_variable_raw = np.load(data_root_lv + '\\validation_autoencoder_input.npy')
validation_label_bf = np.load(data_root_lv + '\\validation_autoencoder_output_stage.npy')
test_latent_variable_raw = np.load(data_root_lv + '\\test_autoencoder_input.npy')
test_label_bf = np.load(data_root_lv + '\\test_autoencoder_output_stage.npy')

In [7]:
training_latent_variable_max = training_latent_variable_raw.max()
training_latent_variable_min = training_latent_variable_raw.min()

In [8]:
training_latent_variable = (training_latent_variable_raw - training_latent_variable_min) / (training_latent_variable_max - training_latent_variable_min)
validation_latent_variable = (validation_latent_variable_raw - training_latent_variable_min) / (training_latent_variable_max - training_latent_variable_min)
test_latent_variable = (test_latent_variable_raw - training_latent_variable_min) / (training_latent_variable_max - training_latent_variable_min)

In [9]:
print(training_latent_variable.shape)
print(training_label_bf.shape)
print(validation_latent_variable.shape)
print(validation_label_bf.shape)
print(test_latent_variable.shape)
print(test_label_bf.shape)

(90, 3, 128, 128)
(90,)
(8, 3, 128, 128)
(8,)
(37, 3, 128, 128)
(37,)


In [10]:
label_one = len(np.where(training_label_bf == 1)[0]) / len(training_label_bf)
label_two = len(np.where(training_label_bf == 2)[0]) / len(training_label_bf)
label_three = len(np.where(training_label_bf == 3)[0]) / len(training_label_bf)
label_four = len(np.where(training_label_bf == 4)[0]) / len(training_label_bf)
print('one : ', label_one)
print('two : ', label_two)
print('three : ', label_three)
print('four  : ', label_four)

one :  0.6333333333333333
two :  0.2222222222222222
three :  0.1111111111111111
four  :  0.03333333333333333


In [11]:
val_label_one = len(np.where(validation_label_bf == 1)[0]) / len(validation_label_bf)
val_label_two = len(np.where(validation_label_bf == 2)[0]) / len(validation_label_bf)
val_label_three = len(np.where(validation_label_bf == 3)[0]) / len(validation_label_bf)
val_label_four = len(np.where(validation_label_bf == 4)[0]) / len(validation_label_bf)
print('one : ', val_label_one)
print('two : ', val_label_two)
print('three : ', val_label_three)
print('four : ', val_label_four)

one :  0.625
two :  0.25
three :  0.125
four :  0.0


In [12]:
test_label_one = len(np.where(test_label_bf == 1)[0]) / len(test_label_bf)
test_label_two = len(np.where(test_label_bf == 2)[0]) / len(test_label_bf)
test_label_three = len(np.where(test_label_bf == 3)[0]) / len(test_label_bf)
test_label_four = len(np.where(test_label_bf == 4)[0]) / len(test_label_bf)
print('one : ', test_label_one)
print('two : ', test_label_two)
print('three : ', test_label_three)
print('four  : ', test_label_four)

one :  0.5405405405405406
two :  0.40540540540540543
three :  0.05405405405405406
four  :  0.0


In [13]:
training_label = np.array([])
for i in training_label_bf:
    if i == 1:
        new_s = 0
    else:
        new_s = 1
    training_label = np.append(training_label, new_s)

In [14]:
validation_label = np.array([])
for i in validation_label_bf:
    if i == 1:
        new_s = 0
    else:
        new_s = 1
    validation_label = np.append(validation_label, new_s)

In [15]:
label_one = len(np.where(training_label == 0)[0]) / len(training_label)
label_three = len(np.where(training_label == 1)[0]) / len(training_label)
print('early_stage : ', label_one)
print('late_stage : ', label_three)

early_stage :  0.6333333333333333
late_stage :  0.36666666666666664


In [16]:
val_label_one = len(np.where(validation_label == 0)[0]) / len(validation_label)
val_label_three = len(np.where(validation_label == 1)[0]) / len(validation_label)
print('early_stage : ', val_label_one)
print('late_stage : ', val_label_three)

early_stage :  0.625
late_stage :  0.375


In [17]:
test_label = np.array([])
for i in test_label_bf:
    if i == 1:
        new_s = 0
    else:
        new_s = 1
    test_label = np.append(test_label, new_s)

In [18]:
label_one = len(np.where(training_label == 0)[0]) / len(training_label)
label_three = len(np.where(training_label == 1)[0]) / len(training_label)
print('early_stage : ', label_one)
print('late_stage : ', label_three)

early_stage :  0.6333333333333333
late_stage :  0.36666666666666664


In [19]:
val_label_one = len(np.where(validation_label == 0)[0]) / len(validation_label)
val_label_three = len(np.where(validation_label == 1)[0]) / len(validation_label)
print('early_stage : ', val_label_one)
print('late_stage : ', val_label_three)

early_stage :  0.625
late_stage :  0.375


In [20]:
test_label_one = len(np.where(test_label == 0)[0]) / len(test_label)
test_label_three = len(np.where(test_label == 1)[0]) / len(test_label)
print('early_stage : ', test_label_one)
print('late_stage : ', test_label_three)

early_stage :  0.5405405405405406
late_stage :  0.4594594594594595


In [21]:
train_label = np.zeros((len(training_label), 2))
for i in range(len(training_label)):
    label = int(training_label[i] - 1)
    train_label[i, label] = 1

In [22]:
val_label = np.zeros((len(validation_label), 2))
for i in range(len(validation_label)):
    label = int(validation_label[i] - 1)
    val_label[i, label] = 1

In [23]:
te_label = np.zeros((len(test_label), 2))
for i in range(len(test_label)):
    label = int(test_label[i] - 1)
    te_label[i, label] = 1

In [24]:
train_x_torch = torch.tensor(training_latent_variable, dtype=torch.float)
train_y_torch = torch.tensor(train_label, dtype=torch.long)

In [25]:
val_x_torch = torch.tensor(validation_latent_variable, dtype=torch.float)
val_y_torch = torch.tensor(val_label, dtype=torch.long)

In [26]:
test_x_torch = torch.tensor(test_latent_variable, dtype=torch.float)
test_y_torch = torch.tensor(te_label, dtype=torch.long)

In [27]:
batch_size = 16
momentum = 0.9
learning_rate = 1e-5
drop_out_ratio = 0.5
num_epochs = 240
num_classes = 2

In [28]:
class Dataset(Dataset):
    def __init__(self, images, label):
        self.labels = label
        self.images = images
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        X = self.images[index]
        y = self.labels[index]
        return X, y

In [29]:
training_set = Dataset(train_x_torch, train_y_torch)
train_loader = DataLoader(training_set, batch_size = batch_size, shuffle=True)
batch_len_train = len(train_loader)

In [30]:
validation_set = Dataset(val_x_torch, val_y_torch)
validation_loader = DataLoader(validation_set, batch_size = batch_size, shuffle=True)
batch_len_val = len(validation_loader)

In [31]:
test_set = Dataset(test_x_torch, test_y_torch)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle=True)
batch_len_val = len(test_loader)

In [32]:
class Model(nn.Module):
    def __init__(self, in_channels, out_channels, out_channels1, out_channels2, flatten_size, out_features1, 
                 out_features2, out_features3, out_features4, out_features5):
        super(Model, self).__init__()
        self.cnn = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1)
        self.cnn1 = nn.Conv2d(out_channels, out_channels1, kernel_size= 3, padding = 1)
        self.cnn2 = nn.Conv2d(out_channels1, out_channels2, kernel_size= 3, padding = 1)
        self.linear1 = nn.Linear(flatten_size, out_features1)
        self.linear2 = nn.Linear(out_features1, out_features2)
        self.linear3 = nn.Linear(out_features2, out_features3)
        self.linear4 = nn.Linear(out_features3, out_features4)
        self.linear5 = nn.Linear(out_features4, out_features5)
        self.BatchNorm = nn.BatchNorm2d(out_channels)
        self.BatchNorm1 = nn.BatchNorm2d(out_channels1)
        self.dropout = nn.Dropout2d(drop_out_ratio)
        self.dropout1 = nn.Dropout(drop_out_ratio)
        
    def forward(self, x):
        output = self.cnn(x)
        output = self.BatchNorm(output)
        output = self.cnn1(output)
        output = self.BatchNorm1(output)
        output = self.dropout(output)
        output = self.cnn2(output)
        output = output.view(output.size(0), -1)
        output = self.dropout1(output)
        output = self.linear1(output)
        output = self.dropout1(output)
        output = self.linear2(output)
        output = self.dropout1(output)
        output = self.linear3(output)      
        output = self.dropout1(output)
        output = self.linear4(output)
        output = self.linear5(output)
        output = F.log_softmax(output, dim = 1)
        return output

In [33]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), data_root_lv + '\\clf_checkpoint_base_line.pt')
        self.val_loss_min = val_loss

In [34]:
flatten_size = 64*128*128

In [35]:
my_model = Model(3, 512, 512, 64, flatten_size, 128, 8, 8, 8, num_classes)
my_model.cuda()

Model(
  (cnn): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (cnn1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (cnn2): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear1): Linear(in_features=1048576, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=2, bias=True)
  (BatchNorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BatchNorm1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout2d(p=0.5, inplace=False)
  (dropout1): Dropout(p=0.5, inplace=False)
)

In [ ]:
early_stopping = EarlyStopping(patience = 10, verbose = True)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(my_model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [ ]:
result = np.array([]).reshape(0, 8)
for epoch in range(num_epochs):
    epoch_loss_train = 0.0
    epoch_train_acc = 0.0
    predicted_train_output = np.array([])
    train_real = np.array([])

    my_model.train()
    for train_x_batch, train_y_batch in train_loader:
        train_x = Variable(train_x_batch).cuda()
        train_y = Variable(train_y_batch).cuda()
        
        optimizer.zero_grad()
        
        train_output = my_model(train_x)
        train_epoch_loss = criterion(train_output, torch.max(train_y, 1)[1])

        train_epoch_loss.backward()
        optimizer.step()

        epoch_loss_train += (train_epoch_loss.data.item() * len(train_x_batch))

        pred = np.argmax(train_output.data.cpu().numpy(), axis = 1)
        true = np.argmax(train_y.data.cpu().numpy(), axis = 1)
        predicted_train_output = np.append(predicted_train_output, pred)
        train_real = np.append(train_real, true)
        correct = len(np.where(pred == true)[0])
        epoch_train_acc += (correct / len(pred))

    train_loss = epoch_loss_train / len(train_x_torch)
    train_acc = len(np.where(predicted_train_output == train_real)[0]) / len(predicted_train_output)


    with torch.no_grad():
        
        '''
        validation
        '''
        epoch_loss_val = 0.0
        epoch_acc_val = 0.0
        predicted_val_output = np.array([])
        val_real = np.array([])
        
        my_model.eval()

        for validation_x_batch, validation_y_batch in validation_loader:
            validation_x = Variable(validation_x_batch).cuda()
            validation_y = Variable(validation_y_batch).cuda()

            validation_output = my_model(validation_x)
            validation_epoch_loss = criterion(validation_output, torch.max(validation_y, 1)[1])

            epoch_loss_val += (validation_epoch_loss.data.item() * len(validation_x_batch))

            pred_val = np.argmax(validation_output.data.cpu().numpy(), axis = 1)
            true_val = np.argmax(validation_y.data.cpu().numpy(), axis = 1)
            correct_val = len(np.where(pred_val == true_val)[0])
            epoch_acc_val += (correct_val / len(pred_val))
            
            predicted_val_output = np.append(predicted_val_output, pred_val)
            val_real = np.append(val_real, true_val)
            

        val_loss = epoch_loss_val / len(val_x_torch)
        val_acc = len(np.where(predicted_val_output == val_real)[0]) / len(predicted_val_output)
        
        '''
        test
        '''
        epoch_loss_test = 0.0
        epoch_acc_test = 0.0
        predicted_test_output = np.array([])
        test_real = np.array([])
        
        my_model.eval()

        for test_x_batch, test_y_batch in test_loader:
            test_x = Variable(test_x_batch).cuda()
            test_y = Variable(test_y_batch).cuda()

            test_output = my_model(test_x)
            test_epoch_loss = criterion(test_output, torch.max(test_y, 1)[1])

            epoch_loss_test += (test_epoch_loss.data.item() * len(test_x_batch))

            pred_test = np.argmax(test_output.data.cpu().numpy(), axis = 1)
            true_test = np.argmax(test_y.data.cpu().numpy(), axis = 1)
            correct_test = len(np.where(pred_test == true_test)[0])
            epoch_acc_test += (correct_test / len(pred_test))
            
            predicted_test_output = np.append(predicted_test_output, pred_test)
            test_real = np.append(test_real, true_test)
            

        test_loss = epoch_loss_test / len(test_x_torch)
        test_acc = len(np.where(predicted_test_output == test_real)[0]) / len(predicted_test_output)
        
        result_list = np.array([str(epoch+1), str(num_epochs), train_loss, train_acc, 
                                val_loss, val_acc, test_loss, test_acc]).reshape(1, 8)
        
    result = np.append(result, result_list, axis = 0)
    early_stopping(val_loss, my_model)
    
    if (epoch + 1) == 1 :
        print('Epoch [{}/{}], Train loss : {:.4f}, Train acc : {:.2f}, Val loss : {:.4f}, Val acc : {:.2f}, Test loss : {:.4f}, Test acc : {:.2f}'. 
              format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))


    if (epoch + 1) % 10 == 0 :
        print('Epoch [{}/{}], Train loss : {:.4f}, Train acc : {:.2f}, Val loss : {:.4f}, Val acc : {:.2f}, Test loss : {:.4f}, Test acc : {:.2f}'. 
              format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))

    if  test_acc > 0.80 :
        print('----------------------------------------------------------------------------------------------------------')
        print('Epoch [{}/{}], Train loss : {:.4f}, Train acc : {:.2f}, Val loss : {:.4f}, Val acc : {:.2f}, Test loss : {:.4f}, Test acc : {:.2f}'. 
              format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))
                                                                                                                      

In [ ]:
result_pd = pd.DataFrame(result, columns = ['num_epoch', 'Epoch', 'train loss', 'train acc', 
                                           'validation loss', 'validation acc', 'test loss', 'test acc'])

In [ ]:
save_root = 'D:\\radiogenomics+radiomics_genomics_LAUD+LSCC'

In [ ]:
result_pd.to_csv(save_root + '\\deep_learning_result_cnn_base_line.csv')

In [ ]:
fname = data_root_lv + '\\clf_checkpoint_base_line.pt'
checkpoint = torch.load(fname)
my_model.load_state_dict(checkpoint)

In [ ]:
with torch.no_grad():
    epoch_loss_train = 0.0
    predicted_train_output = np.array([])
    train_real = np.array([])

    epoch_loss_val = 0.0
    predicted_val_output = np.array([])
    val_real = np.array([])
    
    epoch_loss_test = 0.0
    predicted_test_output = np.array([])
    test_output_log_softmax = np.array([]).reshape(0, num_classes)
    test_real = np.array([])
    
    my_model.eval()
    for train_x_batch, train_y_batch in train_loader:
        train_x = Variable(train_x_batch).cuda()
        train_y = Variable(train_y_batch).cuda()
        
        train_output = my_model(train_x)
        train_epoch_loss = criterion(train_output, torch.max(train_y, 1)[1])

        epoch_loss_train += (train_epoch_loss.data.item() * len(train_x_batch))

        pred = np.argmax(train_output.data.cpu().numpy(), axis = 1)
        true = np.argmax(train_y.data.cpu().numpy(), axis = 1)
        predicted_train_output = np.append(predicted_train_output, pred)
        train_real = np.append(train_real, true)
        correct = len(np.where(pred == true)[0])

    train_loss = epoch_loss_train / len(train_x_torch)
    train_acc = len(np.where(predicted_train_output == train_real)[0]) / len(predicted_train_output)
    
    for validation_x_batch, validation_y_batch in validation_loader:
        validation_x = Variable(validation_x_batch).cuda()
        validation_y = Variable(validation_y_batch).cuda()

        validation_output = my_model(validation_x)
        validation_epoch_loss = criterion(validation_output, torch.max(validation_y, 1)[1])

        epoch_loss_val += (validation_epoch_loss.data.item() * len(validation_x_batch))

        pred_val = np.argmax(validation_output.data.cpu().numpy(), axis = 1)
        true_val = np.argmax(validation_y.data.cpu().numpy(), axis = 1)
        correct_val = len(np.where(pred_val == true_val)[0])
        epoch_acc_val += (correct_val / len(pred_val))

        predicted_val_output = np.append(predicted_val_output, pred_val)
        val_real = np.append(val_real, true_val)
            

    val_loss = epoch_loss_val / len(val_x_torch)
    val_acc = len(np.where(predicted_val_output == val_real)[0]) / len(predicted_val_output)
        
    for test_x_batch, test_y_batch in test_loader:
        test_x = Variable(test_x_batch).cuda()
        test_y = Variable(test_y_batch).cuda()

        test_output = my_model(test_x)
        test_epoch_loss = criterion(test_output, torch.max(test_y, 1)[1])

        epoch_loss_test += (test_epoch_loss.data.item() * len(test_x_batch))

        pred_test = np.argmax(test_output.data.cpu().numpy(), axis = 1)
        true_test = np.argmax(test_y.data.cpu().numpy(), axis = 1)
        correct_test = len(np.where(pred_test == true_test)[0])
        epoch_acc_test += (correct_test / len(pred_test))

        predicted_test_output = np.append(predicted_test_output, pred_test)
        test_output_log_softmax = np.append(test_output_log_softmax, test_output.data.cpu().numpy(), axis = 0)
        test_real = np.append(test_real, true_test)


    test_loss = epoch_loss_test / len(test_x_torch)
    test_acc = len(np.where(predicted_test_output == test_real)[0]) / len(predicted_test_output)

    early_stopping_result_list = np.array([str(epoch+1), str(num_epochs), train_loss, train_acc, 
                            val_loss, val_acc, test_loss, test_acc]).reshape(1, 8)

    
    print('Epoch [{}/{}], Train loss : {:.4f}, Train acc : {:.2f}, Val loss : {:.4f}, Val acc : {:.2f}, Test loss : {:.4f}, Test acc : {:.2f}'. 
              format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc, test_loss, test_acc))


In [ ]:
early_stopping_result_pd = pd.DataFrame(early_stopping_result_list, columns = ['num_epoch', 'Epoch', 'train loss', 'train acc', 
                                           'validation loss', 'validation acc', 'test loss', 'test acc'])

In [ ]:
early_stopping_result_pd

In [ ]:
test_real = test_real.reshape(37, 1)
predicted_test_output = predicted_test_output.reshape(37, 1)

In [ ]:
test_label_output = np.append(test_real, predicted_test_output, axis = 1)

In [ ]:
test_label_output_pd = pd.DataFrame(test_label_output, columns = ['real label', 'predicted label'])

In [ ]:
test_label_output_pd.to_csv(save_root + '\\deep_learning_output_base_line.csv')

In [ ]:
conf_matrix = confusion_matrix(test_real, predicted_test_output)

In [ ]:
conf_matrix

In [ ]:
sensitivity = conf_matrix[0, 0] / conf_matrix.sum(axis = 1)[0]
specificity = conf_matrix[1, 1] / conf_matrix.sum(axis = 1)[1]

In [ ]:
print('sensitivity : ', sensitivity)
print('specificity : ', specificity)

In [ ]:
test_output_probability = np.zeros((len(predicted_test_output), 2))
for i in range(len(predicted_test_output)):
    for j in range(2):
        test_output_probability[i][j] = math.exp(test_output_log_softmax[i][j])

In [ ]:
prob = np.array([])
for i in range(len(test_output_log_softmax)):
    if predicted_test_output[i] == 0:
        prob = np.append(prob, test_output_log_softmax[i][0])
    else :
        prob = np.append(prob, test_output_log_softmax[i][1])

In [ ]:
prob

In [ ]:
fpr, tpr, threshold = roc_curve(predicted_test_output.reshape(37, ), prob, pos_label = 1)

In [ ]:
auc_score = roc_auc_score(predicted_test_output.reshape(37, ), prob)

In [ ]:
auc_score

In [ ]:
plt.plot(fpr,tpr, color = 'red', label='ROC curve (area = %0.2f)' % auc_score)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc=4)
fig = plt.gcf()
fig.savefig(save_root + '\\roc_curve_base_line.png', dpi = fig.dpi)
plt.show()